In [1]:
import sqlite3
import os
from datetime import datetime

db_file = 'library.db'
if os.path.exists(db_file):
    os.remove(db_file)


conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Таблица books
cursor.execute('''
    CREATE TABLE IF NOT EXISTS books (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        author TEXT NOT NULL,
        year_published INTEGER,
        genre TEXT
    )
''')


cursor.executemany('''
    INSERT INTO books (title, author, year_published, genre)
    VALUES (?, ?, ?, ?)
''', [
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"),
    ("1984", "George Orwell", 1949, "Dystopian"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic")
])

cursor.execute("SELECT * FROM books")
print("Все книги:", cursor.fetchall())

cursor.execute("SELECT * FROM books WHERE year_published > 1950")
print("После 1950:", cursor.fetchall())

cursor.execute("SELECT * FROM books WHERE title LIKE 'T%'")
print("Названия с T:", cursor.fetchall())

cursor.execute("SELECT * FROM books ORDER BY year_published ASC")
print("Сортировка по году:", cursor.fetchall())

cursor.execute("SELECT * FROM books ORDER BY title ASC LIMIT 2")
print("Первые 2 по названию:", cursor.fetchall())

cursor.execute("SELECT COUNT(*) FROM books")
print("Всего книг:", cursor.fetchone()[0])

cursor.execute("SELECT genre, COUNT(*) FROM books GROUP BY genre")
print("Книг по жанрам:", cursor.fetchall())


cursor.execute("UPDATE books SET year_published = 1948 WHERE title = '1984'")
cursor.execute("DELETE FROM books WHERE title = 'The Great Gatsby'")

cursor.execute("CREATE INDEX IF NOT EXISTS idx_author ON books(author)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_author_year ON books(author, year_published)")
cursor.execute("PRAGMA index_list('books')")
print("Индексы:", cursor.fetchall())


cursor.execute('''
    CREATE TABLE IF NOT EXISTS logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        action TEXT,
        book_title TEXT,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

cursor.execute('''
    CREATE TRIGGER IF NOT EXISTS log_book_insert
    AFTER INSERT ON books
    BEGIN
        INSERT INTO logs (action, book_title)
        VALUES ('INSERT', NEW.title);
    END;
''')

try:
    conn.execute('BEGIN')
    cursor.execute("INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?)",
                   ("New Book", "Unknown Author", 2020, "Drama"))
    cursor.execute("UPDATE books SET year_published = 2000 WHERE title = 'To Kill a Mockingbird'")
    conn.commit()
except Exception as e:
    conn.rollback()
    print("Ошибка:", e)


def book_age(year):
    return datetime.now().year - year if year else None

conn.create_function("book_age", 1, book_age)

cursor.execute("SELECT title, year_published, book_age(year_published) AS age FROM books")
print("Возраст книг:", cursor.fetchall())

cursor.execute('''
    CREATE VIEW IF NOT EXISTS modern_books AS
    SELECT * FROM books WHERE year_published > 1950
''')

cursor.execute("DROP TABLE IF EXISTS books")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='books'")
print("Таблица существует" if cursor.fetchone() else "Таблица удалена")

conn.close()
if os.path.exists(db_file):
    os.remove(db_file)
    print("База данных удалена")
else:
    print("Файл базы данных не найден")


Все книги: [(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction'), (2, '1984', 'George Orwell', 1949, 'Dystopian'), (3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')]
После 1950: [(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')]
Названия с T: [(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction'), (3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')]
Сортировка по году: [(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction'), (2, '1984', 'George Orwell', 1949, 'Dystopian'), (3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')]
Первые 2 по названию: [(2, '1984', 'George Orwell', 1949, 'Dystopian'), (1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')]
Всего книг: 3
Книг по жанрам: [('Classic', 1), ('Dystopian', 1), ('Fiction', 1)]
Индексы: [(0, 'idx_author_year', 0, 'c', 0), (1, 'idx_author', 0, 'c', 0)]
Ошибка: cannot start a transaction within a transaction
Возраст книг: []
Таблица удалена
База данных удалена
